In [320]:
import numpy as np
import pandas as pd
import torch

import warnings
warnings.filterwarnings("ignore")

# Data exploration

In [117]:
df = pd.read_csv("data/CollegeBasketballPlayers2009-2021.csv", low_memory=False)
df

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,dgbpm,oreb,dreb,treb,ast,stl,blk,pts,Unnamed: 64,Unnamed: 65
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,-1.941150,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,NaN,6.22026
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,-0.247934,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,NaN,3.94375
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,-0.883163,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,NaN,10.92680
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,-0.393459,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,NaN,6.77427
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,-0.668318,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,NaN,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,Trey Patterson,Villanova,BE,2,0.3,60.5,19.3,0.0,0.00,0.0,...,16.018200,0.0000,0.0000,0.0000,0.5000,0.5000,0.0000,0.0000,Pure PG,0.00000
61057,Stavros Polatoglou,Northwestern St.,Slnd,4,1.3,28.3,7.1,0.0,0.00,7.0,...,-4.993820,0.2500,0.0000,0.2500,0.0000,0.0000,0.0000,0.0000,C,0.00000
61058,Sandy Ryan,Tulane,Amer,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-1.126810,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000
61059,Ty Larson,Texas Tech,B12,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-2.380750,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000


In [118]:
df.columns

Index(['player_name', 'team', 'conf', 'GP', 'Min_per', 'Ortg', 'usg', 'eFG',
       'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA',
       'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per',
       'blk_per', 'stl_per', 'ftr', 'yr', 'ht', 'num', 'porpag', 'adjoe',
       'pfr', 'year', 'pid', 'type', 'Rec Rank', 'ast/tov', 'rimmade',
       'rimmade+rimmiss', 'midmade', 'midmade+midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade',
       'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', 'Unnamed: 64', 'Unnamed: 65'],
      dtype='object')

### Biranje atributa

U datasetu postoje 66 atributa za svakog igrača, što je previše. Redukcija prostora atributa će biti vršena u tri koraka:

1. Izbacivanje redundantnih atributa.  _Primer_: Odnos asistencija i izgubljenih lopti (`ast/tov`) je suvišan kada već postoje atributi za broj asistencije i broj izgubljenih lopti
2. Izbacivanje atributa na osnovu iskustvenog zananja. _Primer_: Nepotrebno je da pored škole za koju igra student koristimo kao atribut i konferenciju u kojoj je ta škola
3. Nad ostalim atributima će biti izračunata korelacija sa ciljanim izlazima modela, i biće izabrani najboljih 10

Ovim postupkom će model biti manji, i samim tim će mu biti smanjena mogućnost overfittovanja.

Cilj mreže je predviđanje da li će neki pik biti izabran na draftu. Svi odbirci koji imaju vrednost u `pick` koloni biće označeni kao pozitivni, dok će ostali biti označeni kao negativni

In [215]:
# Choose valuable features
clean_df = df[["GP", "Min_per", "usg", "eFG", "TS_per", "ORB_per", "DRB_per", "AST_per", 
               "TO_per", "FT_per", "twoP_per", "TP_per", "blk_per", "stl_per", "porpag", "adjoe", "adrtg", "bpm"]].copy()
clean_df.dropna(inplace=True)

# BAD DATA LEAKAGE
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(clean_df)
clean_np = scaler.transform(clean_df)

clean_df = pd.DataFrame(clean_np, columns=clean_df.columns)


# Calculate target based on if the player was picked in the draft
clean_df["Target"] = df["pick"].copy().fillna(0) > 0

In [216]:
# Describe features
clean_df.describe()

,GP,Min_per,usg,eFG,TS_per,ORB_per,DRB_per,AST_per,TO_per,FT_per,twoP_per,TP_per,blk_per,stl_per,porpag,adjoe,adrtg,bpm
count,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04,6.101600e+04
mean,5.962336e-17,2.198611e-16,7.182751e-16,3.363130e-16,8.943504e-17,2.748264e-17,-1.695539e-16,-4.564913e-17,-1.117938e-16,-4.341326e-16,6.614466e-17,-5.543109e-17,1.164519e-18,-3.330524e-17,4.099106e-17,-4.844398e-17,1.109321e-15,-3.912783e-17
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00
min,-2.148194e+00,-1.324560e+00,-2.903684e+00,-2.420169e+00,-2.705139e+00,-6.028329e-01,-1.191956e+00,-1.156674e+00,-1.644319e+00,-2.129715e+00,-2.184332e+00,-1.194590e+00,-3.273502e-01,-8.279161e-01,-5.674703e+00,-4.105578e+00,-1.273562e+02,-1.313739e+01
25%,-7.694473e-01,-9.930190e-01,-5.811941e-01,-2.456546e-01,-2.150536e-01,-4.066311e-01,-4.036219e-01,-6.645491e-01,-4.499059e-01,-2.883189e-01,-2.656320e-01,-1.194590e+00,-3.273502e-01,-3.930541e-01,-7.110487e-01,-3.165300e-01,-4.933499e-01,-3.765930e-01
50%,4.123358e-01,-5.187059e-02,-4.576034e-03,1.729394e-01,1.844283e-01,-1.341285e-01,-7.514916e-02,-1.938211e-01,-9.239435e-02,3.009278e-01,1.743898e-01,2.485268e-01,-1.871398e-01,-1.031461e-01,-2.860562e-01,1.748325e-01,3.079122e-02,9.430478e-02
75%,8.062635e-01,8.857128e-01,5.880592e-01,4.773714e-01,4.833294e-01,2.800755e-01,3.190181e-01,4.480808e-01,3.382445e-01,6.544757e-01,5.069644e-01,6.788382e-01,9.328095e-02,2.350799e-01,5.353222e-01,5.594320e-01,5.328688e-01,5.202980e-01
max,1.791083e+00,2.169097e+00,5.104901e+00,5.734260e+00,5.818658e+00,1.712482e+02,1.287894e+02,9.541691e+00,6.480942e+00,1.553077e+00,2.932201e+00,4.053109e+00,1.875546e+02,5.628396e+01,5.913651e+00,1.889007e+01,3.766221e+00,9.068616e+01


In [160]:
# Calculate percentage of True datapoints
value_cnts = clean_df["Target"].value_counts()
print(f"Dataset sadrži {len(clean_df)} odbiraka, od čega su {value_cnts[True] / len(clean_df) * 100:0.2f}% pozitivni.")

Dataset sadrži 61016 odbiraka, od čega su 2.35% pozitivni.


## Podela na trening i test

In [217]:
from sklearn.model_selection import train_test_split

# Do train/test split
train_df, valid_df = train_test_split(clean_df, train_size=.7)

In [123]:
# We can normalize dataset here

# Definisanje modela

Model će se sastojati od duboke neuronske mreže, sa potpuno povezanim slojevima.
Ulaz u mrežu će biti prethodno izabrani atributi, a izlaz će sadžati jedan neuron čiji izlaz predstavlja uverenost modela da će ulazni igrač biti NBA pik.

#### Dataset klasa

Prvi korak predstavlja definisanje klase koja će dohvatati podatke u _batch_-evima tokom treniranja.

In [297]:
from torch.utils.data import Dataset, DataLoader

class NBA_Dataset(Dataset):
    """Class for holding data of NCAA players and if there were picked for the NBA."""

    def __init__(self, df: pd.DataFrame) -> None:
        """Initialize torch Dataset based on pandas Dataframe."""
        super(NBA_Dataset, self).__init__()

        # Take all except the last column as inputs
        self.inputs = df.values[:, :-1].astype(float)

        # Cast Targets column to numpy
        self.targets = df["Target"].values.astype(int)

        # Save dataframe as part of the class
        self.df = df

        # Define dataset length
        self.len = len(df)

    def __getitem__(self, index) -> dict:
        """Return dict with information of datapoint at `index`"""
        # Get input
        input_row = torch.Tensor(self.inputs[index, :])

        # Get output
        target = torch.Tensor([self.targets[index]])

        return {
            "Inputs": input_row,
            "Targets": target,
        }

    def __len__(self):
        """Get dataset length."""
        return self.len

#### Definicija modela

**OVDE SE PISE O TOME STA SVE IMA U MODELU**

In [315]:
import torch.nn as nn

class NBA_Draft_Predictor(nn.Module):
    """Class for prediction NCAA player chances of being piked in NBA draft."""

    def __init__(self, layer_sizes: list, activation_function: nn.Module = None) -> None:
        """Initialize fully-connected model based on input parameters."""
        super(NBA_Draft_Predictor, self).__init__()

        assert len(layer_sizes)

        # Check if activation function is not defined
        if activation_function is None:
            activation_function = nn.ReLU()

        # Define activation function
        self.activation = activation_function

        # Layers init
        layer_list = []

        # Define layer sizes
        for in_size, out_size in zip(layer_sizes[:-1], layer_sizes[1:]):
            layer_list.append(nn.Linear(in_size, out_size))
            # layer_list.append(nn.Tanh())

        # Cast layer list to nn.Module
        self.fc_layers = nn.ModuleList(layer_list)
        # self.layer = nn.Linear(layer_sizes[0], layer_sizes[-1])

    def forward(self, X) -> torch.Tensor:
        """Define network behavior when called on data."""
        # Pass data trough fully-connected layers
        for layer in self.fc_layers:
            X = layer(X)
        
        # Calculate output probability as sigmoid of output
        output_probability = torch.sigmoid(X)

        return output_probability

# Trening

In [299]:
train_dataset = NBA_Dataset(train_df)

train_parameters = {
    "batch_size": 32,
    "shuffle": True,
    "num_workers": 1,
}

training_loader = DataLoader(train_dataset, **train_parameters)

# Validation dataset
valid_dataset = NBA_Dataset(valid_df)

train_parameters = {
    "batch_size": 32,
    "shuffle": False,
    "num_workers": 1,
}

validation_loader = DataLoader(valid_dataset, **train_parameters)

#### Inicijalizacija treninga

**Ovde objasniti zasto smo odabrali Adam optimajzer i BCE loss funkciju**

In [322]:
from torch.nn import BCELoss, CrossEntropyLoss
from torch.optim import Adam, SGD

# Initialize optimizer
model = NBA_Draft_Predictor([18, 1])

# Initialize loss function as Binary Cross Entropy
loss_function = BCELoss()

# Define learning rate
lr = 0.01

# Init Adam optimizer
optimizer = Adam(params=model.parameters(), lr=lr)

# Classification threshold
CLASS_THR = .5

In [323]:
from sklearn.metrics import classification_report

num_of_epochs = 10

for epoch in range(1, num_of_epochs + 1):

    # Run epoch of training
    epoch_loss = 0

    model.train()

    epoch_predictions = []
    epoch_targets = []

    for batch in training_loader:
        
        # Load batch
        X = batch["Inputs"]
        Y = batch["Targets"]

        optimizer.zero_grad()

        # Run forward pass
        probabilities = model(X)

        # Calculate loss
        loss = loss_function(probabilities, Y)
        epoch_loss += loss.item()

        # Update model
        loss.backward()
        optimizer.step()

        # Calculate predictions
        predictions = probabilities >= CLASS_THR

        # Calculate acc
        epoch_predictions.extend(predictions.tolist())
        epoch_targets.extend(Y.tolist())

    epoch_loss /= len(training_loader)

    print(f"Epoch {epoch:03} | Loss: {epoch_loss:.10f}")
    print(classification_report(epoch_targets, epoch_predictions))

Epoch 001 | Loss: 0.1853786727
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     41712
         1.0       0.02      0.02      0.02       999

    accuracy                           0.96     42711
   macro avg       0.50      0.50      0.50     42711
weighted avg       0.95      0.96      0.96     42711

Epoch 002 | Loss: 0.1114300481
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     41712
         1.0       0.00      0.00      0.00       999

    accuracy                           0.98     42711
   macro avg       0.49      0.50      0.49     42711
weighted avg       0.95      0.98      0.97     42711

Epoch 003 | Loss: 0.1113545071
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     41712
         1.0       0.00      0.00      0.00       999

    accuracy                           0.98     42711
   macro avg       0.49      0.50  

KeyboardInterrupt: 